In [4]:
# https://github.com/devopshq/artifactory

import requests.packages.urllib3 as urllib3
urllib3.disable_warnings()

from artifactory import ArtifactoryPath
import os
base_path = "http://127.0.0.1:8086/artifactory"

In [39]:
# Directory Walking
def directory_walk(path):
    
    path = ArtifactoryPath(path)
    return [x for x in path]
        

In [47]:
dir(directory_walk("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact")[0])

['__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__fspath__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__truediv__',
 '_accessor',
 '_cached_cparts',
 '_closed',
 '_cparts',
 '_drv',
 '_flavour',
 '_format_parsed_parts',
 '_from_parsed_parts',
 '_from_parts',
 '_hash',
 '_init',
 '_make_child',
 '_make_child_relpath',
 '_opener',
 '_parse_args',
 '_parts',
 '_pparts',
 '_raise_closed',
 '_raw_open',
 '_root',
 '_str',
 'absolute',
 'anchor',
 'aql',
 'as_posix',
 'as_uri',
 'auth',
 'cert',
 'chmod',
 'copy',
 'create_aql_text',
 'creator',
 'cwd',
 'del_properties',
 'deploy',
 'deploy_deb',
 'deploy_file',
 'drive',
 'exists',
 'expanduser',
 'find_g

In [49]:
# retrieve artifact hashes
print(directory_walk("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact")[0].absolute)

<bound method Path.absolute of ArtifactoryPath('http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.0.txt')>


In [25]:
# User and password
path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test", auth=("admin", "password"))

for p in path:
    print(p)

http://127.0.0.1:8086/artifactory/snet_test/text_artifact


In [26]:
with open('inputs/text_artifact-1.0.txt', 'w') as f:
    f.write('sample data')
with open('inputs/text_artifact-1.1.txt', 'w') as f:
    f.write('sample data')
with open('inputs/text_artifact-1.2.txt', 'w') as f:
    f.write('sample data')


In [27]:
path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/text_artifact", auth=("admin", "password"))

if not path.exists():
    path.mkdir()

In [28]:
# uploading based on simple-layout ([orgPath]/[module]/[module]-[baseRev].[ext])
repo_path = "/snet_test"
org_path = "/tools"
module = "/text_artifact"

repo = base_path+repo_path+org_path+module

path = ArtifactoryPath(repo, auth=("admin", "password"))

if not path.exists():
    path.mkdir()

path.deploy_file("./inputs/text_artifact-1.0.txt")
path.deploy_file("./inputs/text_artifact-1.1.txt")
path.deploy_file("./inputs/text_artifact-1.2.txt")


In [30]:
# retrieve md5 sum and other stats

path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.0.txt")

# Get FileStat
stat = ArtifactoryPath.stat(path)
print(stat)
print(stat.md5)
print(stat.sha1)
print(stat.sha256)
print(stat.ctime)
print(stat.is_dir)
print(stat.size)

ArtifactoryFileStat(ctime=datetime.datetime(2019, 12, 2, 2, 27, 22, 367000, tzinfo=tzutc()), mtime=datetime.datetime(2019, 12, 2, 2, 27, 22, 354000, tzinfo=tzutc()), created_by='admin', modified_by='admin', mime_type='text/plain', size=11, sha1='3579307d55e123bde331b3eefce08090bea3fbe7', sha256='f107aac59dff1d49ebfedb7f03877eaa0297f9a7d3cff26edfc75406f222256d', md5='26e078b87fdaa3206ab8bf63a6096c07', is_dir=False, children=None)
26e078b87fdaa3206ab8bf63a6096c07
3579307d55e123bde331b3eefce08090bea3fbe7
f107aac59dff1d49ebfedb7f03877eaa0297f9a7d3cff26edfc75406f222256d
2019-12-02 02:27:22.367000+00:00
False
11


In [31]:
def download_direct(artifact, dest):      
    arti_path = ArtifactoryPath(artifact, verify=False)
    
    with arti_path.open() as fd:
        with open(os.path.join(dest, arti_path.stem), "wb") as out:
            out.write(fd.read())

def download_artifact(artifact, dest):
    artifact_path = '/'.join([base_path, artifact['repo'], artifact['path'], artifact['name']])
       
    arti_path = ArtifactoryPath(artifact_path, verify=False)
    
    with arti_path.open() as fd:
        with open(os.path.join(dest, artifact['name']), "wb") as out:
            out.write(fd.read())

In [32]:
# Retrieve latest artifact using aql (probably not exposed without credentials)
base_path = "http://127.0.0.1:8086/artifactory"
repo = "snet_test"
org = "tools"
module = "text_artifact"

aql = ArtifactoryPath(base_path, auth=("admin", "password"))

args = ["items.find", {"$and": [
        { "repo": {"$eq": repo} },
        {
            "$or": [
                {"path": {"$match": org+'/'+module}}
            ]
        }
        ]
},
".sort", {"$desc" : ["created"]},
".limit", [1]
]
artifacts_list = aql.aql(*args)

print(artifacts_list)

if artifacts_list:
    download_artifact(artifacts_list[0], 'downloads')

[{'repo': 'snet_test', 'path': 'tools/text_artifact', 'name': 'text_artifact-1.2.txt', 'type': 'file', 'size': 11, 'created': '2019-12-02T02:27:22.582Z', 'created_by': 'admin', 'modified': '2019-12-02T02:27:22.575Z', 'modified_by': 'admin', 'updated': '2019-12-02T02:27:22.583Z'}]


In [33]:
# Retrieve latest artifact using pathlib flavour
path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/", verify=False)

try:
	newest_file = (max(path))
	print(newest_file)
	download_direct(newest_file, 'downloads')
except:
	print('file not found')

http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.2.txt


In [34]:
# Retrieve latest artifact using pathlib flavour
path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/", verify=False)

newest_file = (max(path))
stat = ArtifactoryPath.stat(newest_file)
print(stat)
print(stat.md5)
print(stat.sha1)
print(stat.sha256)
print(stat.ctime)
print(stat.is_dir)
print(stat.size)

ArtifactoryFileStat(ctime=datetime.datetime(2019, 12, 2, 2, 27, 22, 582000, tzinfo=tzutc()), mtime=datetime.datetime(2019, 12, 2, 2, 27, 22, 575000, tzinfo=tzutc()), created_by='admin', modified_by='admin', mime_type='text/plain', size=11, sha1='3579307d55e123bde331b3eefce08090bea3fbe7', sha256='f107aac59dff1d49ebfedb7f03877eaa0297f9a7d3cff26edfc75406f222256d', md5='26e078b87fdaa3206ab8bf63a6096c07', is_dir=False, children=None)
26e078b87fdaa3206ab8bf63a6096c07
3579307d55e123bde331b3eefce08090bea3fbe7
f107aac59dff1d49ebfedb7f03877eaa0297f9a7d3cff26edfc75406f222256d
2019-12-02 02:27:22.582000+00:00
False
11
